# Hello NPU

## Working with NPU in OpenVINO™

#### Table of contents:
- [Introduction](#Introduction)
    - [Install required packages](#Install-required-packages)
- [Checking NPU with Query Device](#Checking-NPU-with-Query-Device)
    - [List the NPU with core.available_devices](#List-the-NPU-with-core.available_devices)
    - [Check Properties with core.get_property](#Check-Properties-with-core.get_property)
    - [Brief Descriptions of Key Properties](#Brief-Descriptions-of-Key-Properties)
- [Compiling a Model on NPU](#Compiling-a-Model-on-NPU)
    - [Download a Model](#Download-and-Convert-a-Model)
    - [Compile with Default Configuration](#Compile-with-Default-Configuration)
    - [Reduce Compile Time through Model Caching](#Reduce-Compile-Time-through-Model-Caching)
        - [UMD Model Caching](#UMD-Model-Caching)
        - [OpenVINO Model Caching](#OpenVINO-Model-Caching)
    - [Throughput and Latency Performance Hints](#Throughput-and-Latency-Performance-Hints)
- [Performance Comparison with benchmark_app](#Performance-Comparison-with-benchmark_app)
    - [NPU vs CPU with Latency Hint](#NPU-vs-CPU-with-Latency-Hint)
        - [Effects of UMD Model Caching](#Effects-of-UMD-Model-Caching)
    - [NPU vs CPU with Throughput Hint](#NPU-vs-CPU-with-Throughput-Hint)
- [Limitations](#Limitations)
- [Conclusion](#Conclusion)
### Installation Instructions

This is a self-contained example that relies solely on its own code.

We recommend  running the notebook in a virtual environment. You only need a Jupyter server to start.
For details, please refer to [Installation Guide](https://github.com/openvinotoolkit/openvino_notebooks/blob/latest/README.md#-installation-guide).

<img referrerpolicy="no-referrer-when-downgrade" src="https://static.scarf.sh/a.png?x-pxid=5b5a4db0-7875-4bfb-bdbd-01698b5b1a77&file=notebooks/hello-npu/hello-npu.ipynb" />


This tutorial provides a high-level overview of working with the NPU device **Intel(R) AI Boost** (introduced with the Intel® Core™ Ultra generation of CPUs) in OpenVINO. It explains some of the key properties of the NPU and shows how to compile a model on NPU with performance hints.

This tutorial also shows example commands for benchmark_app that can be run to compare NPU performance with CPU in different configurations.

## Introduction
[back to top ⬆️](#Table-of-contents:)

The Neural Processing Unit (NPU) is a low power hardware solution which enables you to offload certain neural network computation tasks from other devices, for more streamlined resource management.

Note that the NPU plugin is included in PIP installation of OpenVINO™ and you need to [install a proper NPU driver](https://docs.openvino.ai/2024/get-started/configurations/configurations-intel-npu.html) to use it successfully.

**Supported Platforms**:  
    Host: Intel® Core™ Ultra  
    NPU device: NPU 3720  
    OS: Ubuntu 22.04 (with Linux Kernel 6.6+), MS Windows 11 (both 64-bit)

To learn more about the NPU Device, see the [page](https://docs.openvino.ai/2024/openvino-workflow/running-inference/inference-devices-and-modes/npu-device.html).
    

### Install required packages
[back to top ⬆️](#Table-of-contents:)

In [ ]:
%pip install -q "openvino>=2024.1.0" huggingface_hub

## Checking NPU with Query Device
[back to top ⬆️](#Table-of-contents:)

In this section, we will see how to list the available NPU and check its properties. Some of the key properties will be defined.

### List the NPU with core.available_devices
[back to top ⬆️](#Table-of-contents:)

OpenVINO Runtime provides the ```available_devices``` method for checking which devices are available for inference. The following code will output a list a compatible OpenVINO devices, in which Intel NPU should appear (ensure that the driver is installed successfully). 

In [2]:
import openvino as ov

core = ov.Core()
core.available_devices

['CPU', 'GPU', 'NPU']

### Check Properties with core.get_property
[back to top ⬆️](#Table-of-contents:)

To get information about the NPU, we can use device properties. In OpenVINO, devices have properties that describe their characteristics and configurations. Each property has a name and associated value that can be queried with the ```get_property``` method.

To get the value of a property, such as the device name, we can use the ```get_property``` method as follows:

In [3]:
import openvino.properties as props


device = "NPU"

core.get_property(device, props.device.full_name)

'Intel(R) AI Boost'

Each device also has a specific property called ```SUPPORTED_PROPERTIES```, that enables viewing all the available properties in the device. We can check the value for each property by simply looping through the dictionary returned by ```core.get_property("NPU", props.supported_properties)``` and then querying for that property.

In [ ]:
print(f"{device} SUPPORTED_PROPERTIES:\n")
supported_properties = core.get_property(device, props.supported_properties)
indent = len(max(supported_properties, key=len))

for property_key in supported_properties:
    if property_key not in ("SUPPORTED_METRICS", "SUPPORTED_CONFIG_KEYS", "SUPPORTED_PROPERTIES"):
        try:
            property_val = core.get_property(device, property_key)
        except TypeError:
            property_val = "UNSUPPORTED TYPE"
        print(f"{property_key:<{indent}}: {property_val}")

### Brief Descriptions of Key Properties
[back to top ⬆️](#Table-of-contents:)

Each device has several properties as seen in the last command. Some of the key properties are:
- `FULL_DEVICE_NAME` - The product name of the NPU.
- `PERFORMANCE_HINT` - A high-level way to tune the device for a specific performance metric, such as latency or throughput, without worrying about device-specific settings.
- `CACHE_DIR` - The directory where the OpenVINO model cache data is stored to speed up the compilation time.
- `OPTIMIZATION_CAPABILITIES` - The model data types (INT8, FP16, FP32, etc) that are supported by this NPU.

To learn more about devices and properties, see the [Query Device Properties](https://docs.openvino.ai/2024/openvino-workflow/running-inference/inference-devices-and-modes/query-device-properties.html) page.

## Compiling a Model on NPU
[back to top ⬆️](#Table-of-contents:)

Now, we know the NPU present in the system and we have checked its properties. We can easily use it for compiling and running models with OpenVINO NPU plugin.

## Download a Model
[back to top ⬆️](#Table-of-contents:)

This tutorial uses the `resnet50` model. The `resnet50` model is used for image classification tasks. The model was trained on [ImageNet](https://www.image-net.org/index.php) dataset which contains over a million images categorized into 1000 classes. To read more about resnet50, see the [paper](https://ieeexplore.ieee.org/document/7780459).
As our tutorial focused on inference part, we skip model conversion step. To convert this Pytorch model to OpenVINO IR, [Model Conversion API](https://docs.openvino.ai/2024/openvino-workflow/model-preparation.html) should be used. Please check this [tutorial](https://github.com/openvinotoolkit/openvino_notebooks/blob/latest/notebooks/pytorch-to-openvino/pytorch-to-openvino.ipynb) for details how to convert pytorch model.

In [5]:
from pathlib import Path

# create a directory for resnet model file
MODEL_DIRECTORY_PATH = Path("model")
MODEL_DIRECTORY_PATH.mkdir(exist_ok=True)

model_name = "resnet50"

In [6]:
import huggingface_hub as hf_hub

In [7]:
precision = "FP16"

model_path = MODEL_DIRECTORY_PATH / "ir_model" / f"{model_name}_{precision.lower()}.xml"

model = None
if not model_path.exists():
    hf_hub.snapshot_download("katuni4ka/resnet50_fp16", local_dir=model_path.parent)
    print("IR model saved to {}".format(model_path))
    model = core.read_model(model_path)
else:
    print("Read IR model from {}".format(model_path))
    model = core.read_model(model_path)

Read IR model from model\ir_model\resnet50_fp16.xml


**Note:** NPU also supports `INT8` quantized models.

### Compile with Default Configuration
[back to top ⬆️](#Table-of-contents:)

When the model is ready, first we need to read it, using the `read_model` method. Then, we can use the `compile_model` method and specify the name of the device we want to compile the model on, in this case, "NPU".

In [8]:
compiled_model = core.compile_model(model, device)

### Reduce Compile Time through Model Caching
[back to top ⬆️](#Table-of-contents:)

Depending on the model used, device-specific optimizations and network compilations can cause the compile step to be time-consuming, especially with larger models, which may lead to bad user experience in the application. To solve this **Model Caching** can be used.

Model Caching helps reduce application startup delays by exporting and reusing the compiled model automatically. The following two compilation-related metrics are crucial in this area:

- **First-Ever Inference Latency (FEIL)**:  
  Measures all steps  required to compile and execute a model on the device for the first time. It includes model compilation time, the time required to load and initialize the model on the device and the first inference execution.
- **First Inference Latency (FIL)**:  
  Measures the time required to load and initialize the pre-compiled model on the device and the first inference execution.

In NPU, UMD model caching is a solution enabled by default by the driver. It improves time to first inference (FIL) by storing the model in the cache after compilation (included in FEIL). Learn more about UMD Caching [here](https://docs.openvino.ai/2024/openvino-workflow/running-inference/inference-devices-and-modes/npu-device.html#umd-dynamic-model-caching). Due to this caching, it takes lesser time to load the model after first compilation.

You can also use OpenVINO Model Caching, which is a common mechanism for all OpenVINO device plugins and can be enabled by setting the `cache_dir` property.  
By enabling OpenVINO Model Caching, the UMD caching is automatically bypassed by the NPU plugin, which means the model will only be stored in the OpenVINO cache after compilation. When a cache hit occurs for subsequent compilation requests, the plugin will import the model instead of recompiling it.

#### UMD Model Caching
[back to top ⬆️](#Table-of-contents:)

To see how UMD caching see the following example:

In [30]:
import time
from pathlib import Path

start = time.time()
core = ov.Core()

# Compile the model as before
model = core.read_model(model=model_path)
compiled_model = core.compile_model(model, device)
print(f"UMD Caching (first time) - compile time: {time.time() - start}s")

UMD Caching (first time) - compile time: 3.2854952812194824s


In [31]:
start = time.time()
core = ov.Core()

# Compile the model once again to see UMD Caching
model = core.read_model(model=model_path)
compiled_model = core.compile_model(model, device)
print(f"UMD Caching - compile time: {time.time() - start}s")

UMD Caching - compile time: 2.269814968109131s


#### OpenVINO Model Caching
[back to top ⬆️](#Table-of-contents:)

To get an idea of OpenVINO model caching, we can use the OpenVINO cache as follow

In [19]:
# Create cache folder
cache_folder = Path("cache")
cache_folder.mkdir(exist_ok=True)

start = time.time()
core = ov.Core()

# Set cache folder
core.set_property({props.cache_dir(): cache_folder})

# Compile the model
model = core.read_model(model=model_path)
compiled_model = core.compile_model(model, device)
print(f"Cache enabled (first time) - compile time: {time.time() - start}s")

start = time.time()
core = ov.Core()

# Set cache folder
core.set_property({props.cache_dir(): cache_folder})

# Compile the model as before
model = core.read_model(model=model_path)
compiled_model = core.compile_model(model, device)
print(f"Cache enabled (second time) - compile time: {time.time() - start}s")

Cache enabled (first time) - compile time: 0.6362860202789307s
Cache enabled (second time) - compile time: 0.3032548427581787s


And when the OpenVINO cache is disabled:

In [20]:
start = time.time()
core = ov.Core()
model = core.read_model(model=model_path)
compiled_model = core.compile_model(model, device)
print(f"Cache disabled - compile time: {time.time() - start}s")

Cache disabled - compile time: 3.0127954483032227s


The actual time improvements will depend on the environment as well as the model being used but it is definitely something to consider when optimizing an application. To read more about this, see the [Model Caching docs](https://docs.openvino.ai/2024/openvino-workflow/running-inference/optimize-inference/optimizing-latency/model-caching-overview.html).

### Throughput and Latency Performance Hints
[back to top ⬆️](#Table-of-contents:)

To simplify device and pipeline configuration, OpenVINO provides high-level performance hints that automatically set the batch size and number of parallel threads for inference. The "LATENCY" performance hint optimizes for fast inference times while the "THROUGHPUT" performance hint optimizes for high overall bandwidth or FPS.

To use the "LATENCY" performance hint, add `{hints.performance_mode(): hints.PerformanceMode.LATENCY}` when compiling the model as shown below. For NPU, this automatically minimizes the batch size and number of parallel streams such that all of the compute resources can focus on completing a single inference as fast as possible.

In [21]:
import openvino.properties.hint as hints


compiled_model = core.compile_model(model, device, {hints.performance_mode(): hints.PerformanceMode.LATENCY})

To use the "THROUGHPUT" performance hint, add `{hints.performance_mode(): hints.PerformanceMode.THROUGHPUT}` when compiling the model. For NPUs, this creates multiple processing streams to efficiently utilize all the execution cores and optimizes the batch size to fill the available memory.

In [22]:
compiled_model = core.compile_model(model, device, {hints.performance_mode(): hints.PerformanceMode.THROUGHPUT})

## Performance Comparison with benchmark_app
[back to top ⬆️](#Table-of-contents:)


Given all the different options available when compiling a model, it may be difficult to know which settings work best for a certain application. Thankfully, OpenVINO provides `benchmark_app` - a performance benchmarking tool.

The basic syntax of `benchmark_app` is as follows:

`
benchmark_app -m PATH_TO_MODEL -d TARGET_DEVICE -hint {throughput,cumulative_throughput,latency,none}
`

where `TARGET_DEVICE` is any device shown by the `available_devices` method as well as the MULTI and AUTO devices we saw previously, and the value of hint should be one of the values between brackets.

Note that benchmark_app only requires the model path to run but both device and hint arguments will be useful to us. For more advanced usages, the tool itself has other options that can be checked by running `benchmark_app -h` or reading the [docs](https://docs.openvino.ai/2024/learn-openvino/openvino-samples/benchmark-tool.html). The following example shows us to benchmark a simple model, using a NPU with latency focus:

`benchmark_app -m {model_path} -d NPU -hint latency`

For completeness, let us list here some of the comparisons we may want to do by varying the device and hint used. Note that the actual performance may depend on the hardware used. Generally, we should expect NPU to be better than CPU.  
Please refer to the `benchmark_app` log entries under `[Step 11/11] Dumping statistics report` to observe the differences in latency and throughput between the CPU and NPU..

#### NPU vs CPU with Latency Hint
[back to top ⬆️](#Table-of-contents:)

In [23]:
!benchmark_app -m {model_path} -d CPU -hint latency

[Step 1/11] Parsing and validating input arguments
[ INFO ] Parsing input parameters
[Step 2/11] Loading OpenVINO Runtime
[ INFO ] OpenVINO:
[ INFO ] Build ................................. 2024.1.0-14992-621b025bef4
[ INFO ] 
[ INFO ] Device info:
[ INFO ] CPU
[ INFO ] Build ................................. 2024.1.0-14992-621b025bef4
[ INFO ] 
[ INFO ] 
[Step 3/11] Setting device configuration
[Step 4/11] Reading model files
[ INFO ] Loading model files
[ INFO ] Read model took 14.00 ms
[ INFO ] Original model I/O parameters:
[ INFO ] Model inputs:
[ INFO ]     x (node: x) : f32 / [...] / [1,3,224,224]
[ INFO ] Model outputs:
[ INFO ]     x.45 (node: aten::linear/Add) : f32 / [...] / [1,1000]
[Step 5/11] Resizing model to match image sizes and given batch
[ INFO ] Model batch size: 1
[Step 6/11] Configuring input of the model
[ INFO ] Model inputs:
[ INFO ]     x (node: x) : u8 / [N,C,H,W] / [1,3,224,224]
[ INFO ] Model outputs:
[ INFO ]     x.45 (node: aten::linear/Add) : f32 / [...

In [24]:
!benchmark_app -m {model_path} -d NPU -hint latency

[Step 1/11] Parsing and validating input arguments
[ INFO ] Parsing input parameters
[Step 2/11] Loading OpenVINO Runtime
[ INFO ] OpenVINO:
[ INFO ] Build ................................. 2024.1.0-14992-621b025bef4
[ INFO ] 
[ INFO ] Device info:
[ INFO ] NPU
[ INFO ] Build ................................. 2024.1.0-14992-621b025bef4
[ INFO ] 
[ INFO ] 
[Step 3/11] Setting device configuration
[Step 4/11] Reading model files
[ INFO ] Loading model files
[ INFO ] Read model took 11.51 ms
[ INFO ] Original model I/O parameters:
[ INFO ] Model inputs:
[ INFO ]     x (node: x) : f32 / [...] / [1,3,224,224]
[ INFO ] Model outputs:
[ INFO ]     x.45 (node: aten::linear/Add) : f32 / [...] / [1,1000]
[Step 5/11] Resizing model to match image sizes and given batch
[ INFO ] Model batch size: 1
[Step 6/11] Configuring input of the model
[ INFO ] Model inputs:
[ INFO ]     x (node: x) : u8 / [N,C,H,W] / [1,3,224,224]
[ INFO ] Model outputs:
[ INFO ]     x.45 (node: aten::linear/Add) : f32 / [...

##### Effects of UMD Model Caching
[back to top ⬆️](#Table-of-contents:)

To see the effects of UMD Model caching, we are going to run the benchmark_app and see the difference in model read time and compilation time:

In [25]:
!benchmark_app -m {model_path} -d NPU -hint latency

[Step 1/11] Parsing and validating input arguments
[ INFO ] Parsing input parameters
[Step 2/11] Loading OpenVINO Runtime
[ INFO ] OpenVINO:
[ INFO ] Build ................................. 2024.1.0-14992-621b025bef4
[ INFO ] 
[ INFO ] Device info:
[ INFO ] NPU
[ INFO ] Build ................................. 2024.1.0-14992-621b025bef4
[ INFO ] 
[ INFO ] 
[Step 3/11] Setting device configuration
[Step 4/11] Reading model files
[ INFO ] Loading model files
[ INFO ] Read model took 11.00 ms
[ INFO ] Original model I/O parameters:
[ INFO ] Model inputs:
[ INFO ]     x (node: x) : f32 / [...] / [1,3,224,224]
[ INFO ] Model outputs:
[ INFO ]     x.45 (node: aten::linear/Add) : f32 / [...] / [1,1000]
[Step 5/11] Resizing model to match image sizes and given batch
[ INFO ] Model batch size: 1
[Step 6/11] Configuring input of the model
[ INFO ] Model inputs:
[ INFO ]     x (node: x) : u8 / [N,C,H,W] / [1,3,224,224]
[ INFO ] Model outputs:
[ INFO ]     x.45 (node: aten::linear/Add) : f32 / [...

As you can see from the log entries `[Step 4/11] Reading model files` and `[Step 7/11] Loading the model to the device`, it takes less time to read and compile the model after the initial load.

#### NPU vs CPU with Throughput Hint
[back to top ⬆️](#Table-of-contents:)

In [26]:
!benchmark_app -m {model_path} -d CPU -hint throughput

[Step 1/11] Parsing and validating input arguments
[ INFO ] Parsing input parameters
[Step 2/11] Loading OpenVINO Runtime
[ INFO ] OpenVINO:
[ INFO ] Build ................................. 2024.1.0-14992-621b025bef4
[ INFO ] 
[ INFO ] Device info:
[ INFO ] CPU
[ INFO ] Build ................................. 2024.1.0-14992-621b025bef4
[ INFO ] 
[ INFO ] 
[Step 3/11] Setting device configuration
[Step 4/11] Reading model files
[ INFO ] Loading model files
[ INFO ] Read model took 12.00 ms
[ INFO ] Original model I/O parameters:
[ INFO ] Model inputs:
[ INFO ]     x (node: x) : f32 / [...] / [1,3,224,224]
[ INFO ] Model outputs:
[ INFO ]     x.45 (node: aten::linear/Add) : f32 / [...] / [1,1000]
[Step 5/11] Resizing model to match image sizes and given batch
[ INFO ] Model batch size: 1
[Step 6/11] Configuring input of the model
[ INFO ] Model inputs:
[ INFO ]     x (node: x) : u8 / [N,C,H,W] / [1,3,224,224]
[ INFO ] Model outputs:
[ INFO ]     x.45 (node: aten::linear/Add) : f32 / [...

In [27]:
!benchmark_app -m {model_path} -d NPU -hint throughput

[Step 1/11] Parsing and validating input arguments
[ INFO ] Parsing input parameters
[Step 2/11] Loading OpenVINO Runtime
[ INFO ] OpenVINO:
[ INFO ] Build ................................. 2024.1.0-14992-621b025bef4
[ INFO ] 
[ INFO ] Device info:
[ INFO ] NPU
[ INFO ] Build ................................. 2024.1.0-14992-621b025bef4
[ INFO ] 
[ INFO ] 
[Step 3/11] Setting device configuration
[Step 4/11] Reading model files
[ INFO ] Loading model files
[ INFO ] Read model took 11.50 ms
[ INFO ] Original model I/O parameters:
[ INFO ] Model inputs:
[ INFO ]     x (node: x) : f32 / [...] / [1,3,224,224]
[ INFO ] Model outputs:
[ INFO ]     x.45 (node: aten::linear/Add) : f32 / [...] / [1,1000]
[Step 5/11] Resizing model to match image sizes and given batch
[ INFO ] Model batch size: 1
[Step 6/11] Configuring input of the model
[ INFO ] Model inputs:
[ INFO ]     x (node: x) : u8 / [N,C,H,W] / [1,3,224,224]
[ INFO ] Model outputs:
[ INFO ]     x.45 (node: aten::linear/Add) : f32 / [...

## Limitations
[back to top ⬆️](#Table-of-contents:)

1. Currently, only the models with static shapes are supported on NPU.
2. If the path to the model file includes non-Unicode symbols, such as in Chinese, the model cannot be used for inference on NPU. It will return an error.

## Conclusion
[back to top ⬆️](#Table-of-contents:)

This tutorial demonstrates how easy it is to use NPU in OpenVINO, check its properties, and even tailor the model performance through the different performance hints. 

Discover the power of Neural Processing Unit (NPU) with OpenVINO through these interactive Jupyter notebooks: 
##### Introduction
- [**hello-world**](https://github.com/openvinotoolkit/openvino_notebooks/tree/latest/notebooks/hello-world): Start your OpenVINO journey by performing inference on an OpenVINO IR model.
- [**hello-segmentation**](https://github.com/openvinotoolkit/openvino_notebooks/tree/latest/notebooks/hello-segmentation): Dive into inference with a segmentation model and explore image segmentation capabilities.

##### Model Optimization and Conversion
- [**tflite-to-openvino**](https://github.com/openvinotoolkit/openvino_notebooks/tree/latest/notebooks/tflite-to-openvino): Learn the process of converting TensorFlow Lite models to OpenVINO IR format.
- [**yolov7-optimization**](https://github.com/openvinotoolkit/openvino_notebooks/tree/latest/notebooks/226-yolov7-optimization): Optimize the YOLOv7 model for enhanced performance in OpenVINO.
- [**yolov8-optimization**](https://github.com/openvinotoolkit/openvino_notebooks/tree/main/notebooks/yolov8-optimization): Convert and optimize YOLOv8 models for efficient deployment with OpenVINO.

##### Advanced Computer Vision Techniques
- [**vision-background-removal**](https://github.com/openvinotoolkit/openvino_notebooks/tree/latest/notebooks/vision-background-removal): Implement advanced image segmentation and background manipulation with U^2-Net.
- [**handwritten-ocr**](https://github.com/openvinotoolkit/openvino_notebooks/tree/latest/notebooks/handwritten-ocr): Apply optical character recognition to handwritten Chinese and Japanese text.
- [**vehicle-detection-and-recognition**](https://github.com/openvinotoolkit/openvino_notebooks/tree/latest/notebooks/vehicle-detection-and-recognition): Use pre-trained models for vehicle detection and recognition in images.
- [**vision-image-colorization**](https://github.com/openvinotoolkit/openvino_notebooks/tree/latest/notebooks/vision-image-colorization): Bring black and white images to life by adding color with neural networks.

##### Real-Time Webcam Applications
- [**tflite-selfie-segmentation**](https://github.com/openvinotoolkit/openvino_notebooks/tree/latest/notebooks/tflite-selfie-segmentation): Apply TensorFlow Lite models for selfie segmentation and background processing.
- [**object-detection-webcam**](https://github.com/openvinotoolkit/openvino_notebooks/tree/latest/notebooks/object-detection-webcam): Experience real-time object detection using your webcam and OpenVINO.
- [**pose-estimation-webcam**](https://github.com/openvinotoolkit/openvino_notebooks/tree/latest/notebooks/pose-estimation-webcam): Perform human pose estimation in real-time with webcam integration.
- [**action-recognition-webcam**](https://github.com/openvinotoolkit/openvino_notebooks/tree/latest/notebooks/action-recognition-webcam): Recognize and classify human actions live with your webcam.
- [**style-transfer-webcam**](https://github.com/openvinotoolkit/openvino_notebooks/blob/latest/notebooks/style-transfer-webcam): Transform your webcam feed with artistic styles in real-time using pre-trained models.
- [**3D-pose-estimation-webcam**](https://github.com/openvinotoolkit/openvino_notebooks/tree/latest/notebooks/pose-estimation-webcam): Perform 3D multi-person pose estimation with OpenVINO.
